In [1]:
'065676'

'065676'

In [33]:
# ! pip install jugaad_data, nsetools

import backtrader

from helpers.candlestick import CandlePattern
from helpers.datahandler import *
from helpers.stock_analyser import *

from multiprocessing import Pool
from pathlib import Path

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Defaults

In [34]:
CP = CandlePattern()

current_date = date.today()

pd.set_option('display.max_rows', 100)


# Code

In [5]:
class DataHandler:
    def __init__(self, data_path = './data', check_fresh = True):
        self.present = date.today()
        self.week_num = self.present.strftime("%W")
        
        self.data_path = data_path
        
        if check_fresh:
            self.__fresh()
        self.data = self.read_data()
        self.all_stocks = self.read_data()['all_stocks']
        
         
    def __fresh(self,):
        files = listdir(self.data_path)
        if not len(files):
            raise Exception(f"No CSV data files present at {self.data_path} Download new data for analysis")

        self.data = self.read_data()
        for file in files:
            key, _ , _ = file.split('_')
            self.data['all_stocks'][key] = file
        self.update_data(self.data)
  
    
    def read_data(self, path = './', file = 'data.json'):
        '''
        Write the data in json file
        args:
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file)) as f:
            return json.load(f)


    def update_data(self, updated_data:dict, path:str = './', file:str = 'data.json'):
        '''
        Update the data in the json file
        args:
            updated_data: Dictonary you want to update
            path: Path of the directory
            File: Json Filename
        '''
        with open(join(path,file), 'w') as f:
            json.dump(updated_data,f)
            return True 
    
    
    def open_live_stock_data(self,name:str):
        '''
        Open the fresh stock from the market
        args:
            name: ID of the stock given
        '''
        drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
        return stock_df(symbol=name, from_date = self.present - timedelta(days = 600), to_date = self.present, series="EQ").drop(drop,axis=1)
    
    
    def open_downloaded_stock(self,name:str):
        '''
        Open the Individual stock based on it's Official Term
        args:
            name: Name / ID given to the stock. Example, Infosys is "INFY"
        returns: DataFrame of that stock
        '''
        return pd.read_csv(join(self.data_path,self.all_stocks[name]))

In [86]:
class JournalHandler:
    def __init__(self, key_file:str='client_sec.json',):
        '''
        Read the journal that you have used to track your investments directy from Google Drive using Google API and Credential
        args:
            key_file: json file which holds your secret key
        '''
        scope = 'https://www.googleapis.com/auth/drive' #['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(key_file, scope)
        self.gs = gspread.authorize(creds)
        
        
    def get_journal(self,excel_file_name:str='Finance Journal',working_sheet_name:str='Real Trades'):
        '''
        Open the desired Google Sheet
        args:
            excel_file_name: Name of the Excel File
            working_sheet_name: Name of the Exact you want to open
        '''
        gsheet = self.gs.open(excel_file_name)
        wsheet = gsheet.worksheet(working_sheet_name)
        return pd.DataFrame(wsheet.get_all_records())

In [47]:
class AnalyseStocks(DataHandler):
    def __init__(self, check_fresh = True):
        '''
        args:
            path: Path where all the stock files are saved
        '''
        super().__init__(check_fresh = check_fresh)
        self.registered_stocks = self.read_data()['registered_stocks']
        self.colors = self.read_data()['colors']
        self.rising = {}
    

    def is_ma_eligible(self, df, limit:float, mv = 44, names:tuple = ('DATE','OPEN','CLOSE','LOW','HIGH')):
        '''
        Find the Positive Stocks which are about to rise on the Moving average line
        args:
            df: DataFrame
            limit: Limit difference between average and low/high threshold for selecting stocks
            mv: Moving Average to Consider
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
        '''
        Date, Open, Close, Low, High = names
        Average = f'{str(mv)}-SMA'
        
        stocks = df.sort_index(ascending=False,) # Sort the values else Moving average for new values will be empty
        stocks[Average] = stocks[Close].rolling(mv, min_periods = 1).mean()
        
        last_traded = stocks.iloc[-1,:]
        low, high, open_ , close, avg, symbol = last_traded[Low],last_traded[High],last_traded[Open],last_traded[Close], last_traded[Average], last_traded['SYMBOL']
        
        if close < avg or close < open_ : # if red candle or below Average Line, Discard
            return False
        
        limit = high * 0.05 if not limit else limit # assume limit to be 5% of low
        diff = min(abs(low - avg), (abs(high - avg)), abs(open_ - avg), abs(close - avg)) # min diff between any of the 4 values
        
        if diff <= limit:
            self.rising[symbol] = stocks.iloc[-mv//2:-1,-1].mean() < avg  # whether the moving average is Upward or Downward according to last 44 days moving averages
            return {symbol : round(diff,2)}

        return False
    
    
    def BollingerBands(self,df, mv:int = 44):
        '''
        Calculate the Upper and Lower Bollinger Bands Given on a Moving Average Line
        args:
            df: Stocks Data
            mv: Moving Average Line value
        '''
        ticker = df.sort_index(ascending=False)

        sma = ticker['CLOSE'].rolling(window = mv).mean()
        std = ticker['CLOSE'].rolling(window = mv).std(0)

        upper_bb = sma + std * 2
        lower_bb = sma - std * 2

        ticker['Upper Bollinger'] = upper_bb
        ticker['Lower Bollinger'] = lower_bb

        return ticker.sort_index(ascending=True)
    
    
    def Ichimoku_Cloud(self,df):
        '''
        Get the values of Lines for Ichimoku Cloud
        args:
            df: Dataframe
        '''
        d = df.sort_index(ascending=False)

        # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
        period9_high = d['HIGH'].rolling(window=9).max()
        period9_low = d['LOW'].rolling(window=9).min()
        tenkan_sen = (period9_high + period9_low) / 2


        # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
        period26_high = d['HIGH'].rolling(window=26).max()
        period26_low = d['LOW'].rolling(window=26).min()
        kijun_sen = (period26_high + period26_low) / 2

        # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
        senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

        # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
        period52_high = d['HIGH'].rolling(window=52).max()
        period52_low = d['LOW'].rolling(window=52).min()
        senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

        # The most current closing price plotted 22 time periods behind (optional)
        chikou_span = d['CLOSE'].shift(-26) # Given at Trading View.

        d['blue_line'] = tenkan_sen
        d['red_line'] = kijun_sen
        d['cloud_green_line_a'] = senkou_span_a
        d['cloud_red_line_b'] = senkou_span_b
        d['lagging_line'] = chikou_span
        return d.sort_index(ascending=True)
    
    
    def _is_ichi(self,df, index:str = 'nifty_50'):
        '''
        Get All available IchiMoku available stocks
        args:
            index: Index to Open From
        '''
        count  = 0
        current = df.iloc[0,:]
        if current['cloud_green_line_a'] < current['LOW'] and current['cloud_red_line_b'] < current['LOW']: # Cloud Below
            count += 1
        if df.loc[26,'lagging_line'] > df.loc[26,'HIGH']: # Lagging Line
            count += 1
        if df.loc[1,'blue_line'] <= df.loc[1,'red_line'].min() and current['blue_line'] >= current['red_line']: # Cross Over
            count += 1
        
        return count
    
    
    def update_eligible(self, limit:float):
        '''
        Save all Eligible stocks for the current week
        args:
            limit: Distance between MA line and the Min(open,close,low,high)
        '''
        self.eligible = {}
        for key in self.registered_stocks:
            result = self.is_ma_eligible(self.open_downloaded_stock(key), limit = limit)
            if result:
                self.eligible.update(result)
        return self.eligible
    
    
    def get_RSI(self, data, periods:int = 14, Close:str = 'CLOSE', ema:bool = True, return_df:bool = False):
        '''
        Calculate RSI: Relative Strength Index
        args:
            data: Pandas DataFrame
            periods: Length of Moving Window
            Close: Name of the column which contains the Closing Price
            ema: Whether to use Exponential Moving Average instead of Simple
            return_df: Whether to return the whole Dataframe. NOTE: It'll have the last value removed
        '''
        df = data.copy()
        if df.iloc[0,0] > df.iloc[1,0]: # if the first Date entry [0,0] is > previous data entry [1,0] then it is in descending order, then reverse it for calculation
            df.sort_index(ascending=False, inplace = True)

        close_delta = df['CLOSE'].diff()

        # Make two series: one for lower closes and one for higher closes
        up = close_delta.clip(lower=0)
        down = -1 * close_delta.clip(upper=0)

        if ema: # Use exponential moving average
            ma_up = up.ewm(com = periods - 1, min_periods = periods).mean()
            ma_down = down.ewm(com = periods - 1, min_periods = periods).mean()

        else: # Use simple moving average
            ma_up = up.rolling(window = periods,).mean()
            ma_down = down.rolling(window = periods,).mean()

        rsi = ma_up / ma_down
        rsi = 100 - (100/(1 + rsi))
        df['RSI'] = rsi
        df.sort_index(ascending=True, inplace = True)

        if return_df:
            return df

        return df.iloc[0,-1]
    
    
    
    def get_ATR(self, df, window:int=14, names:tuple = ('OPEN','CLOSE','LOW','HIGH'), return_df:bool = False):
        '''
        Get the Average True Range. Concept of Volatility
        args:
            df: Pandas Data Frame
            window: Rolling window or the period you want to consider
            names: Column names showing ('OPEN','CLOSE','LOW','HIGH') in the same order
            return_df: Whether to return the whole Df or the latest value
        '''
        Open, Close, Low, High = names
        data = df.copy()
        if data.iloc[0,0] > data.iloc[1,0]: # if the first Date entry [0,0] is > previous data entry [1,0] then it is in descending order, then reverse it for calculation
            data.sort_index(ascending=False, inplace = True)


        high_low = data[High] - data[Low]
        high_close = np.abs(data[High] - data[Close].shift())
        low_close = np.abs(data[Low] - data[Close].shift())

        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)

        ATR = true_range.rolling(window).mean()
        
        data['ATR'] = ATR
        data.sort_index(ascending=True, inplace = True)

        if return_df:
            return data

        return data.iloc[0,-1]
            
    
    def plot_candlesticks(self,df, names = ('DATE','OPEN','CLOSE','LOW','HIGH'), mv:list = [44,100,200]):
        '''
        Plot a candlestick on a given dataframe
        args:
            df: DataFrame
            names: Tuple of column names showing ('DATE','OPEN','CLOSE','LOW','HIGH')
            mv: Moving Averages
        '''
        stocks = df.copy()
        Date, Open, Close, Low, High = names
        colors = sample(self.colors,len(mv))
        stocks.sort_index(ascending=False, inplace = True)  # Without reverse, recent rolling mean will be either NaN or equal to the exact value
    

        candle = go.Figure(data = [go.Candlestick(x = stocks[Date], name = 'Trade',
                                                       open = stocks[Open], 
                                                       high = stocks[High], 
                                                       low = stocks[Low], 
                                                       close = stocks[Close]),])
        for i in range(len(mv)):
            stocks[f'{str(mv[i])}-SMA'] = stocks[Close].rolling(mv[i], min_periods = 1).mean()
            candle.add_trace(go.Scatter(name=f'{str(mv[i])} MA',x=stocks[Date], y=stocks[f'{str(mv[i])}-SMA'], 
                                             line=dict(color=colors[i], width=1.1)))

        candle.update_xaxes(
            title_text = 'Date',
            rangeslider_visible = True,
            rangeselector = dict(
                buttons = list([
                    dict(count = 1, label = '1M', step = 'month', stepmode = 'backward'),
                    dict(count = 6, label = '6M', step = 'month', stepmode = 'backward'),
                    dict(count = 1, label = 'YTD', step = 'year', stepmode = 'todate'),
                    dict(count = 1, label = '1Y', step = 'year', stepmode = 'backward'),
                    dict(step = 'all')])))

        candle.update_layout(autosize = False, width = 1400, height = 600,
                             title = {'text': f"{stocks['SYMBOL'][0]} | {self.all_stocks[stocks['SYMBOL'][0]]}",'y':0.97,'x':0.5,
                                      'xanchor': 'center','yanchor': 'top'},
                             margin=dict(l=30,r=30,b=30,t=30,pad=2),
                             paper_bgcolor="lightsteelblue",)

        candle.update_yaxes(title_text = 'Price in Rupees', tickprefix = u"\u20B9" ) # Rupee symbol
        candle.show()

# Download New Files Data

In [8]:
def get_stocks(name):
    try:
        df = stock_df(symbol=name, from_date = current_date - timedelta(days = 600), to_date = current_date, series="EQ").drop(drop,axis=1)
        df['DATE'] = pd.to_datetime(df['DATE'])
        ID, NAME, _ = all_stocks[name].split('_')
        save = f"./data/{ID}_{NAME}_{str(current_date)}.csv"
        df.to_csv(save,index=None)
    except Exception as e:
        print(e)
        pass


 
path = './data'
AS = AnalyseStocks(check_fresh = False)
all_stocks = AS.all_stocks

drop = ['SERIES','PREV. CLOSE','VWAP','VOLUME','VALUE','NO OF TRADES']
current_date = date.today()
stocks = set(all_stocks.keys()) - set([i.split('_')[0] for i in listdir('./data')]) 

pool = Pool(4)
results = pool.map(get_stocks,stocks)
pool.close()
pool.join()

# Investing

In [9]:
class Investing(AnalyseStocks):
    def __init__(self,rolling_mean:int=44):
        '''
        args:
            rolling_mean: Rolling Simple Mean to calulate
        '''
        super().__init__()
        self.rm = rolling_mean
        self._eligible = None
        self.data = self.read_data()
        self.indices = {'nifty_50': 'Nifty 50','nifty_100':'Nifty 100','nifty_200':'Nifty 200','nifty_500':'Nifty 500'}
        self.all_ichi = None
        self.picked = None
        self._old_budget = -1
        self.diff = -1
      

    def _get_all_ichi(self,budget:float, index:str='nifty_500', refit = False):
        '''
        Get all Stocks who are almost perfect for Ichimoku execution
        args:
            budget: Your Budget
            index: Which Index to Search
        '''
        if self.all_ichi and  (not refit):
            return self.all_ichi
        
        all_ichi = {}
        data = self.data[index] if index else self.all_stocks
        for name in data:
            df = self.Ichimoku_Cloud(self.open_downloaded_stock(name))
            count = self._is_ichi(df,index) 
            if count and df.loc[0,'HIGH'] < budget:
                all_ichi[name] = count
                
        self.all_ichi = all_ichi
        return self.all_ichi
    
    
    def highlight_falling(self, s, column:str):
        '''
        Highlight The rows where average is falling
        args:
            s: Series
            column: Column name(s)
        '''
        is_max = pd.Series(data=False, index=s.index)
        is_max[column] = s.loc[column] == True
        return ['' if is_max.any() else 'background-color: #f7a8a8' for v in is_max]
    
    
    def get_index(self,symbol:str):
        '''
        Get the Index of the symbol from nifty 50,100,200,500
        args:
            symbol: Name /  ID od the company on NSE
        '''
        for index in self.indices.keys():
            if symbol in self.data[index]:
                return self.indices[index]
        return 'Other'
    
    
    def calculate(self, budget, High:str = 'HIGH', Close:str = 'CLOSE', delta:float = 1, nifty:str = 'nifty_200', diff = 13, show_only:bool=True):
        '''
        Pick Stocks based on all available and which are within your budget
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            show_only: If you want to see formatted part only
            diff: Max Allowed Distance between Min(close,open,low,high) and High
        ''' 
        refit = True if (self._old_budget < budget or self.diff != diff) else False
        self._old_budget = budget
        self.diff = diff
        ichi = self._get_all_ichi(budget ,refit = refit)
        
        if (not self._eligible) or refit:
            self._eligible  = self.update_eligible(limit = diff)
            
        keys = set(self._eligible.keys()).intersection(set(self.data[nifty])) if nifty else list(self._eligible.keys()) 
        if not len(keys):
            warnings.warn('No matching Stocks Found. Increase Distance or Nifty Index')
            return None
        
        values = []
        one_can = []
        two_can = []
        three_can = []
        atr = []
        rsi = []
        
        for key in keys:
            try:
                df = self.open_downloaded_stock(key)
            except Exception as e:
                    print('Exception Opening: ',key)
                
            if df.loc[0,High] + delta > budget:
                del self._eligible[key]
                
            else:
                values.append(df.iloc[0,:])
                one_can.append(CP.find_name(df.loc[0,'OPEN'],df.loc[0,'CLOSE'],df.loc[0,'LOW'],df.loc[0,'HIGH']))
                two_can.append(CP.double_candle_pattern(df))
                three_can.append(CP.triple_candle_pattern(df))
                rsi.append(self.get_RSI(df))
                atr.append(self.get_ATR(df))
                
                
        columns = df.columns
        df = pd.DataFrame(values,columns=columns,index = range(len(values)))
        df = df.merge(pd.DataFrame({'SYMBOL':self._eligible.keys(), 'Diff':self._eligible.values()}),on='SYMBOL')
        
        df['Rising'] = df['SYMBOL'].apply(lambda x: self.rising[x]) # Get Rising or Falling
        df['Ichi'] = df['SYMBOL'].apply(lambda x: ichi[x] if ichi.get(x) else 0)
        df['RSI'] = rsi
        df['ATR'] = atr
        
        df['Triple Candle'] = three_can
        df['Double Candle'] = two_can
        df['Recent Candle'] = one_can
        
        df['Index'] = df['SYMBOL'].apply(lambda x: self.get_index(x)) # Get Rising or Falling

        self.picked = df.sort_values('Diff',ascending = True)
        if show_only:
            return self.picked.style.apply(self.highlight_falling, column=['Rising'], axis=1) # set style
        else:
            return self.picked
        
        
    def show_full_stats(self, budget, risk, High = 'HIGH', Close = 'CLOSE', delta:float=1, diff:float = 13, nifty:str = 'nifty_500',):
        '''
        Show Extra Stats
        args:
            budget: Total available budget. Stocks under this budget will be considered only
            risk: How much risk you want to take
            High: Column name which show High
            Close: Column Name which shows last closing price
            delta: Value above the Last Highest Traded Price
            nifty: nifty index to consider
            diff: MAx Allowed Difference between Line and the Price
        '''
        self.picked = self.calculate(budget, High, Close, delta, nifty = nifty, diff = diff, show_only = False)
        
        expec_change = []
        max_risk = []
        
         
        if not isinstance(self.picked, pd.DataFrame):
            return 'No match found. Increase Budget / Diff / Risk / Nifty Index '
        
        pic = self.picked.copy()
        for index in pic.index:
            name = pic.loc[index,'SYMBOL']
            result = self.get_particulars(name,budget,risk)
            if result:
                expec_change.append(result['Target %'])
                max_risk.append(result['Max loss on this config'])
            else:
                expec_change.append(np.inf)
                max_risk.append(np.inf)

        pic['Expected Change %'] = expec_change
        pic['Max Config Risk'] = max_risk
        return pic
        
        
    def get_particulars(self, name, budget:float, max_loss_capacity:float, risk_to_reward_ratio:float=2, Low:str = 'LOW', High:str = 'HIGH', delta:float = 0.0033, plot_candle:bool = False):
        '''
        Display the particulars of a trade before buying
        args:
            name: name of the particular stock
            loss_capacity: How much loss you can survive at the end of day PER SHARE. Total capacity will be No of shares * per share loss capacity
            risk_to_reward_ratio: How much profit you want to have. It is twice of loss_capacity per share for 44 Moving average
            budget : How much you have for investing purpose
            Low: Column name which describes LOW of the previous trade
            High =  Column name which describes High of the previous trade
            delta: A min amount above which you'll buy
            plot_candle: Plot the candlestick for the stock
        '''   
        df = self.open_downloaded_stock(name)
        if risk_to_reward_ratio > 2:
            warnings.warn(f"Don't be greedy with risk to reward ratio of {risk_to_reward_ratio}. Stick to system")
            
        if  max_loss_capacity > 0.011 * budget:
            warnings.warn(f"You are risking {round(max_loss_capacity/budget,2)*100}% of your total PORTFOLIO. Going Like this, you'll lose Rs {max_loss_capacity*15} in 15 Trades. Try keeping it less than 1.1% @ Rs {round(budget*0.011,2)}.")
        
        
        buy_delta = df.loc[0,High] * delta  
        sell_delta = min(df.loc[:1,Low].values) * delta
        
        risk = max_loss_capacity # per share
        
        entry = df.loc[0,High] + buy_delta # Last Day MAX + Delta
        stop_loss = min(df.loc[:1,Low].values) - sell_delta # Min of the last 2 
        
        max_loss = entry - stop_loss
        stop_loss_perc = round((max_loss / entry) *100,2)
        diff = entry - stop_loss
        quantity = min(risk // diff , budget // entry)
        profit = risk_to_reward_ratio * diff
        profit_perc = round((profit/entry)*100,2)
        target = round(entry + profit,2)
        
        
        if plot_candle:
            AnalyseStocks().plot_candlesticks(df)
    
        if budget < entry:
            warnings.warn(f"Budget for {name} should be a minimum of Rs. {entry}")
            return None
        
        if quantity < 1:
            r = round(risk + (diff - risk), 2)
            warnings.warn(f"Risk should be atleast {r} for you to afford {name}")
            return None
            
        return {'Buying Price':round(entry,2),'Stop-Loss %': stop_loss_perc,'Target %':profit_perc,'Quantity':quantity,'Stop-Loss Price':stop_loss,'Trigger Price':target,
                'Risk Per Share':round(diff,2),'Profit Per Share':round(profit,2),'Max loss on this config':round(quantity*diff,2),
                'Max Gain on this config': round(quantity*profit,2),'Price To Profit Ratio': round(entry / (diff*2 ),2), 'Index':self.get_index(name),}
    
In = Investing()

In [10]:
df = In.show_full_stats(10000,100.10, diff = None, nifty='nifty_200')
df = df[(df['Expected Change %'] <=10) & (df['RSI']<70) & (df['Ichi']>=1)]
df

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 199.42 for you to afford NAVINFLUOR
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 476.77 for you to afford PIIND
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 102.9 for you to afford GRASIM
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 255.55 for you to afford INDIAMART
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 204.88 for you to afford PEL
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ipykernel_launcher.py:220: UserWarning: Risk should be atleast 130.86 for you to afford DALBHARAT
/home/admin1/anaconda3/envs/finance/lib/p

,DATE,OPEN,HIGH,LOW,LTP,CLOSE,52W H,52W L,SYMBOL,Diff,Rising,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
3,2021-08-02,206.45,207.50,205.45,207.20,207.30,239.20,163.35,ITC,0.05,False,1,52.051089,3.632143,V Pattern,Unknown,Hammer,Nifty 50,4.09,97.98
1,2021-08-02,2093.25,2122.10,2079.00,2095.35,2101.55,2460.00,1785.40,IPCALAB,0.33,False,1,47.441075,60.175000,Unknown,Unknown,Hammer,Nifty 200,5.35,56.96
8,2021-08-02,145.05,148.35,144.75,147.80,147.45,157.30,76.35,INDHOTEL,2.05,True,1,53.672137,4.375000,Unknown,Unknown,Unknown,Nifty 200,7.54,95.43
19,2021-08-02,434.75,436.70,431.80,434.35,434.90,444.40,175.50,SBIN,4.53,True,2,56.123561,8.407143,Unknown,Unknown,Hammer,Nifty 50,4.05,97.49
22,2021-08-02,852.95,871.25,852.95,862.05,860.45,934.25,395.00,CUMMINSIND,5.89,True,2,52.933580,19.417857,Unknown,Unknown,Inverted Hammer,Nifty 200,8.11,70.90
12,2021-08-02,1728.00,1776.00,1725.00,1776.00,1771.55,1792.95,1043.00,TITAN,5.98,True,1,63.377046,32.189286,V Pattern,Unknown,Bullish,Nifty 50,8.56,76.31
20,2021-08-02,777.25,787.00,771.00,786.85,785.30,832.00,376.00,BHARATFORG,7.05,True,2,53.801292,17.464286,Unknown,Unknown,Unknown,Nifty 200,5.54,87.56
0,2021-08-02,1038.00,1044.45,1029.95,1040.05,1040.60,1139.50,720.90,SBICARD,19.55,False,3,62.561339,27.728571,Unknown,Unknown,Hammer,Nifty 100,6.89,72.19
32,2021-08-02,771.00,780.95,766.55,774.50,774.65,833.95,273.45,TATACHEM,20.34,True,2,58.764782,18.253571,Unknown,Unknown,Hammer,Nifty 200,8.90,69.71
6,2021-08-02,588.20,595.40,582.25,595.00,592.25,601.80,268.00,WIPRO,26.73,True,2,66.391547,15.492857,Unknown,Unknown,Hammer,Nifty 50,5.70,85.18


In [23]:
In.get_particulars('TITAN',10000,100.10,)

{'Buying Price': 1781.86,
 'Stop-Loss %': 4.28,
 'Target %': 8.56,
 'Quantity': 1.0,
 'Stop-Loss Price': 1705.55304,
 'Trigger Price': 1934.48,
 'Risk Per Share': 76.31,
 'Profit Per Share': 152.62,
 'Max loss on this config': 76.31,
 'Max Gain on this config': 152.62,
 'Price To Profit Ratio': 11.68,
 'Index': 'Nifty 50'}